In [19]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [87]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#### In this cell, we define a function make_graph. It takes a dataframe with stock data (Dtae and Closing price), dataframe with revenue data (Date and Revenue incurred) and name of the stock.

In [20]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

#### Extracting Stock Data using yfinance library.The yfinance is a Python library with a user-friendly interface for downloading historical market data from Yahoo Finance. It lets you get historical stock prices, dividends, and other financial data for stocks, exchange-traded funds (ETFs), and other securities.

In [ ]:
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period = 'max')
tesla_data.reset_index(inplace = True)
tesla_data.head()

In [ ]:
gamestop = yf.Ticker('GME')
gamestop_data = tesla.history(period = 'max')
gamestop_data.reset_index(inplace = True)
gamestop_data.head()

#### Web Scraping to extract revenue data and covert it into a dataframe to be able to pass it as a make_graph function argument.

In [129]:
url_tesla = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
url_gamestop = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'

##### Define a function web_scrape that takes the url and name of the stock as arguments and returns a dataframe conatining Date and Revenue as its columns ( such that it can be used as an argument for make_graph function defined above)

These are the following tags which are used while creating HTML tables.

* &lt;table&gt;: This tag is a root tag used to define the start and end of the table. All the content of the table is enclosed within these tags. 


* &lt;tr&gt;: This tag is used to define a table row. Each row of the table is defined within this tag.

* &lt;td&gt;: This tag is used to define a table cell. Each cell of the table is defined within this tag. You can specify the content of the cell between the opening and closing <td> tags.

* &lt;th&gt;: This tag is used to define a header cell in the table. The header cell is used to describe the contents of a column or row. By default, the text inside a <th> tag is bold and centered.

* &lt;tbody&gt;: This is the main content of the table, which is defined using the <tbody> tag. It contains one or more rows of <tr> elements.

In [136]:
def web_scrape(url, name):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data, 'html.parser')
    
    ''' This for loop, loops through all the tables and matches a table with table header 
    as name + Quarterly Revenue and stores the index of the matched table in 'i' '''
    
    for i in range(len(soup.find_all('table'))):
        if name + ' Quarterly Revenue' in soup.find_all('table')[i].find('th'): 
            break
        else: continue

    revenue_table = pd.DataFrame(columns = ["Date", "Revenue"])

    ''' First we find the table table with index 'i' (i.e,. with header as name + Quarterly Revenue ) , find its body and 
    return all the rows and then loop through each row to extract date and revenue'''
    
    for row in soup.find_all('table')[i].find('tbody').find_all('tr'):
        col = row.find_all('td')
        date = col[0].text
        revenue = col[1].text
    
        revenue_table = pd.concat([revenue_table, pd.DataFrame({'Date' : [date], 'Revenue' : [revenue]})], ignore_index=True)

    revenue_table["Revenue"] = revenue_table['Revenue'].str.replace(',|\$',"")
    revenue_table.dropna(inplace=True)
    revenue_table = revenue_table[revenue_table['Revenue'] != ""]

    return revenue_table

In [132]:
tesla_revenue = web_scrape(url_tesla, 'Tesla')
gamestop_revenue = web_scrape(url_gamestop, 'GameStop')

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [ ]:
make_graph(gamestop_data, gamestop_revenue, 'GameStop')